### Буду менять названия переменных по возможности на транслит, чтобы не было будто ctrl+c, ctrl+v

In [1]:
# Подключаем библиотеки

import pandas as pd
import numpy as np

In [2]:
# Подключаем библиотеку для коннекта с базой sql и коннектимся к базе "Демо"

import psycopg2
soedinenie = psycopg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='r128256512'")

In [3]:
# Формируем запрос к базе "Демо" на языке SQL

zapros = """
WITH analiz AS
(	
	SELECT
		aircrafts.model::jsonb->>'en' AS samolet,
		aircrafts.range AS max_rass,
		flights.flight_id AS ID_poleta,
		dep_air.airport_name::jsonb->'ru' AS nachalo_poleta,
		arr_air.airport_name::jsonb->'ru' AS konec_poleta,
		ACOS((SIN(PI()*dep_air.coordinates[1]/180)*SIN(PI()*arr_air.coordinates[1]/180))+(COS(PI()*dep_air.coordinates[1]/180)*COS(PI()*arr_air.coordinates[1]/180)*COS(PI()*dep_air.coordinates[0]/180-PI()*arr_air.coordinates[0]/180)))*6371 AS kilometraj
	FROM
		aircrafts_data AS aircrafts
	JOIN
		flights
		ON aircrafts.aircraft_code = flights.aircraft_code
	JOIN
		airports_data AS dep_air
		ON flights.departure_airport = dep_air.airport_code
	JOIN
		airports_data AS arr_air
		ON flights.arrival_airport = arr_air.airport_code
)
SELECT
	ID_poleta,
	samolet,
	max_rass,
	nachalo_poleta,
	konec_poleta,
	kilometraj,
	AVG(kilometraj) OVER(PARTITION BY samolet) AS srednee_km,
	SUM (tick.amount) AS summa_za_reis,
	CASE 
		WHEN samolet LIKE '%Boeing%' THEN 'Boeing'
		WHEN samolet LIKE '%Sukhoi%' THEN 'Sukhoi'
		WHEN samolet LIKE '%Airbus%' THEN 'Airbus'
	END AS marka
FROM
	analiz
JOIN
	ticket_flights AS tick
	ON analiz.ID_poleta = tick.flight_id
WHERE
	samolet LIKE '%Boeing%' 
	OR samolet LIKE '%Sukhoi%'
	OR samolet LIKE '%Airbus%'
GROUP BY
	ID_poleta,
	samolet,
	max_rass,
	ID_poleta,
	nachalo_poleta,
	konec_poleta,
	kilometraj
ORDER BY
	ID_poleta
"""

# Отправляем запрос и вытаскиваем данные в переменную

tablica = pd.read_sql(zapros, con=soedinenie)
tablica

C:\Users\Руслан\AppData\Local\Temp\ipykernel_6972\2557928005.py:62: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tablica = pd.read_sql(zapros, con=soedinenie)


,id_poleta,samolet,max_rass,nachalo_poleta,konec_poleta,kilometraj,srednee_km,summa_za_reis,marka
0,1,Airbus A321-200,5600,Домодедово,Пулково,666.883841,1024.622756,922100.0,Airbus
1,2,Airbus A321-200,5600,Домодедово,Пулково,666.883841,1024.622756,1075800.0,Airbus
2,5,Airbus A321-200,5600,Домодедово,Пулково,666.883841,1024.622756,449400.0,Airbus
3,6,Airbus A321-200,5600,Домодедово,Пулково,666.883841,1024.622756,847400.0,Airbus
4,9,Airbus A321-200,5600,Домодедово,Пулково,666.883841,1024.622756,891100.0,Airbus
...,...,...,...,...,...,...,...,...,...
24673,65297,Sukhoi Superjet-100,3000,Беслан,Внуково,1474.333621,1154.592627,250100.0,Sukhoi
24674,65298,Sukhoi Superjet-100,3000,Беслан,Внуково,1474.333621,1154.592627,29400.0,Sukhoi
24675,65299,Sukhoi Superjet-100,3000,Беслан,Внуково,1474.333621,1154.592627,176500.0,Sukhoi
24676,65300,Sukhoi Superjet-100,3000,Беслан,Внуково,1474.333621,1154.592627,441300.0,Sukhoi


In [4]:
# Показываем количество строк и столбцов

tablica.shape

(24678, 9)

In [5]:
# Показываем типы данных по столбцам

tablica.dtypes

id_poleta           int64
samolet            object
max_rass            int64
nachalo_poleta     object
konec_poleta       object
kilometraj        float64
srednee_km        float64
summa_za_reis     float64
marka              object
dtype: object

In [6]:
# Считаем количество уникальных значений по строковой переменной

print(tablica['marka'].unique())
print(tablica['marka'].unique().size)

['Airbus' 'Boeing' 'Sukhoi']
3


In [7]:
# Считаем частоту встречающихся значений по строковой переменной

tablica['marka'].value_counts()

marka
Sukhoi    13980
Boeing     5937
Airbus     4761
Name: count, dtype: int64

In [8]:
# Считаем минимум, среднюю, медиану, максимум по числовой переменной

print('Минимальный километраж - ', tablica['kilometraj'].min())
print('Средний километраж - ', tablica['kilometraj'].mean())
print('Максимальный километраж - ', tablica['kilometraj'].max())
print('Медиана по километражу - ', np.median(tablica['kilometraj']))

Минимальный километраж -  304.3019636114598
Средний километраж -  1682.9512922473236
Максимальный километраж -  6774.7292687675135
Медиана по километражу -  1168.3448688604883


In [9]:
# Сравним найденные значения со значениями функции .describe()

tablica['kilometraj'].describe()

count    24678.000000
mean      1682.951292
std       1556.967041
min        304.301964
25%        629.860554
50%       1168.344869
75%       1908.399432
max       6774.729269
Name: kilometraj, dtype: float64

In [10]:
# Получается всё верно!